# Data Preparation

Data preparation plan:

1. Load the transaction data
2. Merge the transaction data with the fleet register datasets
3. Clean the columns and remove all unnecessary transactions
4. Add the coordinates of the transactions based on the merchant name
5. Add the true fuel prices to the transactions
6. Create a second dataset with the total fuel consumption per vehicle based on the tracking dataset

# 1. Load the transaction data

In [20]:
import pandas as pd
import openpyxl
import os

In [21]:
# Load in the transaction dataset
raw_trans = pd.read_csv(os.path.join("..", "data", "2021 Monthly Full Transactions.csv"))

In [22]:
# Check dataset size
raw_trans.shape

(207702, 8)

In [23]:
# Check the data structure
raw_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207702 entries, 0 to 207701
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Transaction Date    207693 non-null  object 
 1   REG_NUM             207693 non-null  object 
 2   Merchant Name       207675 non-null  object 
 3   Purchase Category   207693 non-null  object 
 4   No. of Litres       207693 non-null  float64
 5   Transaction Amount  207702 non-null  float64
 6   Make                207693 non-null  object 
 7   Model               207693 non-null  object 
dtypes: float64(2), object(6)
memory usage: 12.7+ MB


# 2. Merge the transaction data with the fleet register datasets

## 2.1 Load the fleet register datasets

In [24]:
full_register = pd.read_csv(os.path.join("..", "data", "full_fleet_register.csv"), sep=';')

colnames = ['NEW REG  NO.', 'VEHICLE MAKE', 'MODEL DERIVATIVE', 'DEPARTMENT', 'RATE CARD CATEGORY']

full_register = full_register[colnames]

full_register = full_register.rename(columns={'NEW REG  NO.': 'Reg No'})

In [25]:
white = pd.read_csv(os.path.join("..", "data", "white fleet data.csv"), sep=';')

colnames = ['Reg No', 'Old Reg', 'Site', 'District']

white = white[colnames]

white = white.rename(columns={'Rental ': 'Rental'})

In [26]:
ems = pd.read_csv(os.path.join("..", "data", "EMS Fleet Data Sep 2022.csv"), sep=';')

colnames = ['Reg No', 'Old Reg', 'Site', 'District']

# Select columns
ems = ems[colnames]

# Rename the rental column from " Rental " to "Rental"
ems = ems.rename(columns={' Rental ': 'Rental'})

## 2.2 Combine the white and ems fleet register datasets

In [27]:
white_ems = pd.concat([white, ems], ignore_index=True)

In [28]:
cols = ['Site', 'District']

# Reshaping the DataFrame for 'New' IDs
new_regs = white_ems.melt(id_vars=cols, 
                  value_vars=['Reg No'], 
                  var_name='Type', 
                  value_name='Reg').drop(columns=['Type'])

# Reshaping the DataFrame for 'Old' IDs
old_regs = white_ems.melt(id_vars=cols, 
                  value_vars=['Old Reg'], 
                  var_name='Type', 
                  value_name='Reg').drop(columns=['Type'])

# Concatenating the two DataFrames
reshaped_white_ems = pd.concat([new_regs, old_regs]).sort_values(by='Reg').reset_index(drop=True)

## 2.3 Merge the datasets

In [29]:
raw_data = pd.merge(raw_trans, full_register, how='left', left_on='REG_NUM', right_on='Reg No')
raw_data = raw_data.drop(columns=['Reg No'])

In [30]:
# left join df and white on REG_NUM and Reg No
raw_data = pd.merge(raw_data, reshaped_white_ems, how='left', left_on='REG_NUM', right_on='Reg')

# drop the Reg column
raw_data = raw_data.drop(columns=['Reg'])

# check for null values
print(raw_data.isnull().sum())

Transaction Date       1674
REG_NUM                1674
Merchant Name          1692
Purchase Category      1674
No. of Litres          1674
Transaction Amount        0
Make                   1674
Model                  1674
VEHICLE MAKE          27750
MODEL DERIVATIVE      27631
DEPARTMENT            27631
RATE CARD CATEGORY    27631
Site                  94759
District              95109
dtype: int64


## 3. Clean the columns and remove all unnecessary transactions

## 3.1 Find all the vehicles in the transaction dataset that are not in the fleet register dataset

In [31]:
# Remove all rows with null where the "MODEL DERIVATIVE" is null - vehicles under invstigation and not included in analysis
clean_data = raw_data.dropna(subset=['MODEL DERIVATIVE'])

# check for null values
print(clean_data.isnull().sum())

Transaction Date          0
REG_NUM                   0
Merchant Name            17
Purchase Category         0
No. of Litres             0
Transaction Amount        0
Make                      0
Model                     0
VEHICLE MAKE            119
MODEL DERIVATIVE          0
DEPARTMENT                0
RATE CARD CATEGORY        0
Site                  74969
District              75094
dtype: int64


In [32]:
clean_data.shape

(182075, 14)

## 3.2 Vehicle make

In [33]:
# Check which "VEHICLE MAKE" and "MODEL DERIVATIVE" are null
clean_data[clean_data['VEHICLE MAKE'].isnull()]

,Transaction Date,REG_NUM,Merchant Name,Purchase Category,No. of Litres,Transaction Amount,Make,Model,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,District
7604,2021-04-14,GGF298EC,TOTAL SAVOY MTHATHA,FUEL,300.00,4953.00,MAN,26.423 DFTF T/T C/C,NaN,M.A.N 27.463 DFTF INTARDER,TRANSPORT,MANAGED MAINTENANCE,NaN,NaN
25827,2021-05-06,GGF298EC,SASOL SUTHERLAND,FUEL,198.84,3241.25,MAN,26.423 DFTF T/T C/C,NaN,M.A.N 27.463 DFTF INTARDER,TRANSPORT,MANAGED MAINTENANCE,NaN,NaN
78467,2021-07-21,GGW518EC,AFRICAN FRONT T/A CP MOTORS,FUEL,500.00,8435.00,TRAILER,1 AXLE TANKER,NaN,SEMI TRL - 1 AXLE TANKER,RURAL DEVELOPMENT AND AGRARIAN REFORM,MANAGED MAINTENANCE,NaN,NaN
78468,2021-07-27,GGW518EC,AFRICAN FRONT T/A CP MOTORS,FUEL,533.40,9000.00,TRAILER,1 AXLE TANKER,NaN,SEMI TRL - 1 AXLE TANKER,RURAL DEVELOPMENT AND AGRARIAN REFORM,MANAGED MAINTENANCE,NaN,NaN
100415,2021-08-03,GGW518EC,AFRICAN FRONT T/A CP MOTORS,FUEL,500.00,8435.00,TRAILER,1 AXLE TANKER,NaN,SEMI TRL - 1 AXLE TANKER,RURAL DEVELOPMENT AND AGRARIAN REFORM,MANAGED MAINTENANCE,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206464,2022-01-27,GGW518EC,TOTAL BIZANA,FUEL,500.00,9580.00,TRAILER,1 AXLE TANKER,NaN,SEMI TRL - 1 AXLE TANKER,RURAL DEVELOPMENT AND AGRARIAN REFORM,MANAGED MAINTENANCE,NaN,NaN
206465,2022-01-27,GGW518EC,TOTAL BIZANA,FUEL,950.00,18202.00,TRAILER,1 AXLE TANKER,NaN,SEMI TRL - 1 AXLE TANKER,RURAL DEVELOPMENT AND AGRARIAN REFORM,MANAGED MAINTENANCE,NaN,NaN
206466,2022-01-14,GGW594EC,BISHO MOTORS,FUEL,400.01,7580.20,TANKER,DIESEL,NaN,2013 TANKER DIESEL,RURAL DEVELOPMENT AND AGRARIAN REFORM,MANAGED MAINTENANCE,NaN,NaN
206467,2022-01-04,GGW595EC,EDWARDS GARAGE & SERVICE STATION,FUEL,1000.00,19190.00,TANKER,DIESEL,NaN,2013 TANKER DIESEL,RURAL DEVELOPMENT AND AGRARIAN REFORM,MANAGED MAINTENANCE,NaN,NaN


In [34]:
# Remove the trucks and tankers from the dataset
clean_data = clean_data[~clean_data['VEHICLE MAKE'].isnull()]

# check for null values
print(clean_data.isnull().sum())

Transaction Date          0
REG_NUM                   0
Merchant Name            17
Purchase Category         0
No. of Litres             0
Transaction Amount        0
Make                      0
Model                     0
VEHICLE MAKE              0
MODEL DERIVATIVE          0
DEPARTMENT                0
RATE CARD CATEGORY        0
Site                  74850
District              74975
dtype: int64


In [35]:
# Check the unique VEHICLE MAKE values
clean_data['VEHICLE MAKE'].unique()

array(['ISUZU', 'VOLKSWAGEN', 'HINO', 'TOYOTA', 'NISSAN', 'UD TRUCKS',
       'FORD', 'HYUNDAI', 'MAHINDRA', 'MERCEDES-BENZ', 'AUDI', 'Ford',
       'BMW', 'CHEVROLET', 'Nissan ', 'OPEL', 'FIAT', 'ISUZU ', 'MASSEY',
       'JEEP', 'TATA', 'MITSUBISHI FUSO', 'IVECO', 'LANDROVER',
       'ROSENBAUER', 'NISSAN DIESEL'], dtype=object)

In [36]:
# Create a list of makes to check
makes_to_check = ['HINO', 'UD TRUCKS', 'IVECO', 'MITSUBISHI', 'TATA', 'NISSAN', 'ROSENBAUER', 'MITSUBISHI FUSO']

# For each of the makes to remove, display the unique models
pd.DataFrame(clean_data[clean_data['VEHICLE MAKE'].isin(makes_to_check)].groupby('VEHICLE MAKE')['MODEL DERIVATIVE'].unique())

,MODEL DERIVATIVE
VEHICLE MAKE,
HINO,"[HINO 300 814 LWB, HINO 300 915 LWB (BA3) , HI..."
IVECO,"[IVECO 35S12V12 F/C P/V, IVECO 50C15V15 F/C P/..."
MITSUBISHI FUSO,"[FUSO CANTER FG6-136 SWA F/C C/, MITSUBISHI CA..."
NISSAN,"[NISSAN HARDBODY NP300 2.5 TDI, NISSAN 2.5D SE..."
ROSENBAUER,[PANTHER 6X6 CA5 36.710RHD FIRE ENGINE]
TATA,"[TATA INDICA 1.4 DLS, TATA INDICA 1.4 LSI]"
UD TRUCKS,"[UD CWE330 (E22) MIX BASIC 250H, UD CWE330(E22..."


In [37]:
models_to_remove = ['IVECO', 'MITSUBISHI FUSO', 'ROSENBAUER', 'UD TRUCKS', 'HINO']

# Remove all of the makes to remove
clean_data = clean_data[~clean_data['VEHICLE MAKE'].isin(models_to_remove)]

In [38]:
# Display all unique model derivatives
clean_data['MODEL DERIVATIVE'].unique()

array(['D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER',
       'D-MAX 250 HO 4X4 CREW CAB HI-RIDER',
       'VW Crafter 50 2.0 TDI 120 KW XLWB 23S',
       'Hilux DC 2.4GD6 4X4 SRX MT', 'Hilux DC 2.4GD6 RB SRX MT',
       'D-MAX 300 4X4 REGULAR CAB LX',
       'D-MAX 250C REGULAR CAB  FLEETSIDE  REGUL',
       'Isuzu D-Max 250 HO 4X4 Regular Cab Hi-Rider Ambulance',
       '250 D/C Hi-Ride GEN 6', 'TOYOTA QUANTUM 2.7 14 SEAT',
       'TOYOTA QUANTUM 2.5 D-4D SESFIK', 'VW Golf & GTI 2.0 DSG 169 KW',
       'VW POLO VIVO 1.6 SEDAN', 'VW Amarok 2.0 Trendline - ACD7 D/C',
       'Amarok 2.0 Bi Tdi 132 kw Auto Highline 4 motion DC',
       'VW Crafter Panel van Ambulance',
       'VW Crafter 50 2.0 TDI XLWB Ambulance',
       'VW CRAFTER 50 2.0 BITDI HR 120', 'NISSAN HARDBODY NP300 2.5 TDI',
       'NISSAN 2.5D SE+SC 4X4 ABS', 'NISSAN HARDBODY NP300 2.5TDI H',
       'NISSAN 2.4 DC 4X4 HR+ABS', 'NISSAN HARDBODY NP300 2.4I HI-',
       'Nissan K36 NP300 2.4 Hi Rider D/C (Response)',
       'NISSAN N

In [39]:
clean_data.head()

,Transaction Date,REG_NUM,Merchant Name,Purchase Category,No. of Litres,Transaction Amount,Make,Model,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,District
0,2021-04-01,GGA344EC,ULUNDI FILLING STATION,FUEL,43.06,695.45,ISUZU,DMAX 250 HO DC HR 4X,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,libode,OR Tambo
1,2021-04-01,GGA345EC,SB DELTA,FUEL,31.02,478.17,ISUZU,DMAX 250 HO DC HR 4X,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,JANSENVILLE,Sarah Baartman
2,2021-04-01,GGA345EC,SB DELTA,FUEL,58.95,908.52,ISUZU,DMAX 250 HO DC HR 4X,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,JANSENVILLE,Sarah Baartman
3,2021-04-01,GGA346EC,MOUNT FLETCHER MOTORS EASTERN CAPE,FUEL,58.61,963.66,ISUZU,DMAX 250 HO DC HR 4X,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,ALIWAL NORTH,Alfred Nzo
4,2021-04-01,GGA346EC,MOUNT FLETCHER MOTORS EASTERN CAPE,FUEL,60.02,986.76,ISUZU,DMAX 250 HO DC HR 4X,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,ALIWAL NORTH,Alfred Nzo


In [40]:
# Convert all model derivatives to uppercase
clean_data['MODEL DERIVATIVE'] = clean_data['MODEL DERIVATIVE'].str.upper()

In [41]:
# Check the unique model derivatives that contain "UD"
clean_data[clean_data['MODEL DERIVATIVE'].str.contains('UD')]['MODEL DERIVATIVE'].unique()

array(['AUDI 716-Q7 3.0D-183-QA8-7S', 'AUDI Q7 3.0 TDI V6 QUATTRO TIP',
       'AUDI Q5 4.0L TDI QUATTRO', 'NISSAN UD 40A F/C C/C',
       'AUDI Q5 40TDI QUATTRO'], dtype=object)

In [42]:
# Remove all rows where the model derivative contains "TRUCK", "NPR", "NQR", "MASSEY" or "AXOR"
clean_data = clean_data[~clean_data['MODEL DERIVATIVE'].str.contains('TRUCK|NPR|NQR|MASSEY|AXOR|BUS|FTR')]

# Remove all rows where the model derivative is 'NISSAN UD 40A F/C C/C'
clean_data = clean_data[~clean_data['MODEL DERIVATIVE'].str.contains('NISSAN UD 40A F/C C/C')]

In [43]:
# Remove the make and model columns
clean_data = clean_data.drop(columns=['Make', 'Model'])

In [44]:
print(clean_data.isnull().sum())

Transaction Date          0
REG_NUM                   0
Merchant Name            16
Purchase Category         0
No. of Litres             0
Transaction Amount        0
VEHICLE MAKE              0
MODEL DERIVATIVE          0
DEPARTMENT                0
RATE CARD CATEGORY        0
Site                  73898
District              74023
dtype: int64


## 3.2 Purchase Category

In [45]:
# Remove all non-fuel transactions
clean_data = clean_data[clean_data['Purchase Category'] == 'FUEL']

In [46]:
clean_data.shape

(175933, 12)

In [47]:
# Remove the purchase category column
clean_data = clean_data.drop(columns=['Purchase Category'])

In [48]:
print(clean_data.isnull().sum())

Transaction Date          0
REG_NUM                   0
Merchant Name            14
No. of Litres             0
Transaction Amount        0
VEHICLE MAKE              0
MODEL DERIVATIVE          0
DEPARTMENT                0
RATE CARD CATEGORY        0
Site                  73603
District              73727
dtype: int64


## 3.3 Merchant Names

In [49]:
from fuzzywuzzy import process
import pandas as pd

def find_similar_names(name, names, threshold=95):
    """
    Finds similar names in a list of names.
    
    :param name: The name to find similarities to.
    :param names: The list of names to search in.
    :param threshold: The similarity threshold (0-100). Names with a similarity score above this threshold will be considered similar.
    :return: A list of similar names.
    """
    similar_names = process.extractBests(name, names, score_cutoff=threshold)
    # Filter out exact matches (similarity score of 100)
    return [sim_name for sim_name, score in similar_names if score < 100]

def consolidate_names(names_dict):
    """
    Consolidates similar names into a single representative name.

    :param names_dict: A dictionary where keys are original names and values are lists of similar names.
    :return: A dictionary mapping each name (including similar ones) to a single representative name.
    """
    consolidated_dict = {}
    for original_name, similar_names in names_dict.items():
        # Include the original name itself in the mapping
        consolidated_dict[original_name] = original_name
        for similar_name in similar_names:
            # Map similar names to the original name
            consolidated_dict[similar_name] = original_name
    return consolidated_dict

def replace_names(df, names_map, var='Merchant Name'):
    """
    Replace names in the dataframe using a mapping dictionary.

    :param df: The DataFrame containing the names.
    :param names_map: A dictionary mapping each name to a representative name.
    :return: DataFrame with replaced names.
    """
    # Replace names in the DataFrame using the mapping
    df[var] = df[var].map(names_map).fillna(df[var])
    return df

In [113]:
# Extract unique merchant names
unique_names = clean_data['Merchant Name'].unique()

# Remove null values
unique_names = unique_names[~pd.isnull(unique_names)]

# Dictionary to hold each name and its similar names
similar_names_dict = {}

# Iterate over each unique name
for name in unique_names:
    # Find similar names
    similar_names = find_similar_names(name, unique_names)
    # Store in the dictionary
    similar_names_dict[name] = similar_names

# Print the results, excluding exact matches
for name, similarities in similar_names_dict.items():
    if similarities:  # Only print if there are non-exact matches
        print(f"{name}: {similarities}")

AMALINDA MOTORS: ['AMALINDA MOTORS CC']
SAFARI MOTORS: ['SHELL SAFARI MOTORS']
SHELL ULTRA CITY (QUEENSTOWN): ['SHELL ULTRA CITY QUEENSTOWN']
TOTAL DEVEREUX: ['TOTAL RA DEVEREUX']
TOTAL RA DEVEREUX: ['TOTAL DEVEREUX']
Q & Q MOTORS MOUNT FRERE: ['MOUNT FRERE MOTORS']
LEO MOTORS KIRKWOOD: ['KIRKWOOD MOTORS KIRKWOOD']
ELLMORES MOTOR SERVICES GRAHAMSTOWN: ['GRAHAMSTOWN MOTOR SERVICES']
GRAHAMSTOWN MOTOR SERVICES: ['ELLMORES MOTOR SERVICES GRAHAMSTOWN']
MOTHERWELL SERVICE STATION: ['MOTHERWELL SERVICE STATION PTY LTD']
BP @ GREENACRES: ['BP GREENACRES']
KIRKWOOD MOTORS KIRKWOOD: ['LEO MOTORS KIRKWOOD']
BLOEMS MOTORS: ['BLOMS MOTORS']
BELL MOTORS QUEENSTOWN: ['BELLS MOTORS QUEENSTOWN']
MOUNT FRERE MOTORS: ['Q & Q MOTORS MOUNT FRERE']
HADAF VULSTASIE WEST: ['HADAF VULSTASIE EAST']
MOTHERWELL SERVICE STATION PTY LTD: ['MOTHERWELL SERVICE STATION']
SHELL ULTRA CITY QUEENSTOWN: ['SHELL ULTRA CITY (QUEENSTOWN)']
AMALINDA MOTORS CC: ['AMALINDA MOTORS']
HADAF VULSTASIE EAST: ['HADAF VULSTASIE WEST'

In [114]:
# Consolidate similar names into a single representative name for each group
names_map = consolidate_names(similar_names_dict)

# Replace names in the DataFrame
clean_data = replace_names(clean_data, names_map)

# View the replaced DataFrame
clean_data.head()

,Transaction Date,REG_NUM,Merchant Name,No. of Litres,Transaction Amount,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,District
0,2021-04-01,GGA344EC,ULUNDI FILLING STATION,43.06,695.45,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,libode,OR Tambo
1,2021-04-01,GGA345EC,SB DELTA,31.02,478.17,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,JANSENVILLE,Sarah Baartman
2,2021-04-01,GGA345EC,SB DELTA,58.95,908.52,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,JANSENVILLE,Sarah Baartman
3,2021-04-01,GGA346EC,MOUNT FLETCHER MOTORS EASTERN CAPE,58.61,963.66,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,ALIWAL NORTH,Alfred Nzo
4,2021-04-01,GGA346EC,MOUNT FLETCHER MOTORS EASTERN CAPE,60.02,986.76,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,ALIWAL NORTH,Alfred Nzo


In [115]:
# Convert the merchant names to camel case
clean_data['Merchant Name'] = clean_data['Merchant Name'].str.title()

## 3.4 Transaction Amount

In [116]:
# Check for negative transaction amounts - check these for later
clean_data[clean_data['Transaction Amount'] < 0]

,Transaction Date,REG_NUM,Merchant Name,No. of Litres,Transaction Amount,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,District
219,2021-04-01,GGZ844EC,NaN,0.0,-798.20,TOYOTA,TOYOTA QUANTUM 2.5 D-4D SESFIK,HEALTH,CATEGORY 15: Minibus 14-16 seater,Raymond Mhlaba LSA Office Admin *P,Amathole
967,2021-04-04,GGA416EC,NaN,0.0,-656.24,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,GFMS RENTAL DESK,CATEGORY 8: LDV Ambulance,NaN,NaN
2920,2021-04-07,GGV361EC,NaN,0.0,-1114.23,FORD,FORD RANGER 2.2TDCI XL P/U D/C,HEALTH,CATEGORY 13: LDV 4x4 D/Cab,Dir: HIV/AIDS & STD MANGT *P,Alfred Nzo
7373,2021-04-13,GGZ971EC,NaN,0.0,-10006.90,ISUZU,KB 250C REGULAR CAB FLEETSIDE,HEALTH,CATEGORY 5: LDV 4x2 1 ton,St Elizabeth Hospital *P,OR Tambo
7374,2021-04-13,GGZ971EC,NaN,0.0,-1006.90,ISUZU,KB 250C REGULAR CAB FLEETSIDE,HEALTH,CATEGORY 5: LDV 4x2 1 ton,St Elizabeth Hospital *P,OR Tambo
7375,2021-04-13,GGZ971EC,NaN,0.0,-1006.90,ISUZU,KB 250C REGULAR CAB FLEETSIDE,HEALTH,CATEGORY 5: LDV 4x2 1 ton,St Elizabeth Hospital *P,OR Tambo
9579,2021-04-16,GGA337EC,NaN,0.0,-590.35,VOLKSWAGEN,VW CRAFTER PANEL VAN AMBULANCE,EMERGENCY MEDICAL SERVICES,CATEGORY 18: Ambulances Large p/van,CATHCART,Amathole


## 3.5 Department

In [117]:
# Check for all unique Department values
clean_data['DEPARTMENT'].unique()

array(['EMERGENCY MEDICAL SERVICES', 'HEALTH',
       'GFMS RENTAL DESK          ', 'TRANSPORT',
       'PUBLIC WORKS & INFRASTRUCTURE', 'SOCIAL DEVELOPMENT',
       'COOPERATIVE GOVERNANCE AND TRADITIONAL AFFAIRS',
       'HUMAN SETTLEMENTS', 'GFMS POOL', 'EDUCATION',
       'PROVINCIAL TREASURY', 'RURAL DEVELOPMENT AND AGRARIAN REFORM',
       'ECONOMIC DEVELOPMENT AND ENVIRONMENTAL AFFAIRS',
       'OFFICE OF THE PREMIER', 'SPORTS, RECREATION, ARTS AND CULTURE   ',
       'COMMUNITY SAFETY  ', 'GFMS RENTAL DESK', 'GFMS RENTAL DESK '],
      dtype=object)

In [118]:
# Trim the white space from the end of the Department values
clean_data['DEPARTMENT'] = clean_data['DEPARTMENT'].str.strip()

## 3.6 Rate Card Category

In [119]:
# Display all unique rate card categories
clean_data['RATE CARD CATEGORY'].unique()

array(['CATEGORY 8: LDV Ambulance', 'CATEGORY 33: LDV D/cab Response',
       'CATEGORY 29: Bus 20-23 seater', 'CATEGORY 13: LDV 4x4 D/Cab',
       'CATEGORY 6: LDV 4x2 D/Cab', 'CATEGORY 10: LDV Rescue',
       'CATEGORY 5: LDV 4x2 1 ton ', 'MANAGED MAINTENANCE',
       'CATEGORY 15: Minibus 14-16 seater',
       'CATEGORY 2: Sedan Traffic 1400-1600cc',
       'CATEGORY 1: Sedan 1400-1600cc',
       'CATEGORY 18: Ambulances Large p/van',
       'CATEGORY 46: 14-16 Seater Patient Transport',
       'CATEGORY 32: LDV Heavy Rescue',
       'CATEGORY 11: LDV 4x4 1 ton light', 'CATEGORY 34: LDV Traffic',
       'CATEGORY 38: LDV Mortuary-Hydraulic', 'CATEGORY 9: LDV Clinic',
       'CATEGORY 12: LDV 4x4 1 ton heavy',
       'CATEGORY 17: Ambulances  standard p/van',
       'CATEGORY 7: LDV Mortuary', 'CATEGORY 4: LDV 4x2 0.5 ton',
       'CATEGORY 3: Hatch 2000 Paramedic',
       'CATEGORY 39:  Wheel chair Minibus 14-16 seater',
       'CATEGORY 16: Minibus p/van',
       'CATEGORY 36: Traf

In [120]:
# Check how many transactions were for the 'MANAGED MAINTENANCE' rate card category
clean_data[clean_data['RATE CARD CATEGORY'] == 'MANAGED MAINTENANCE'].shape

(20482, 11)

In [121]:
# Select a few random transactions for the managed maintenance rate card category
clean_data[clean_data['RATE CARD CATEGORY'] == 'MANAGED MAINTENANCE'].sample(10)

,Transaction Date,REG_NUM,Merchant Name,No. of Litres,Transaction Amount,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,District
25140,2021-05-28,GGF123EC,Sherwoods Humansdorp,51.64,828.85,TOYOTA,TOYOTA HILUX 2.4 GD-6 SR 4X4,EDUCATION,MANAGED MAINTENANCE,NaN,NaN
13257,2021-04-21,GGF138EC,Stigo'S Service Centre,32.04,544.75,TOYOTA,TOYOTA COROLLA QUEST 1.6,EDUCATION,MANAGED MAINTENANCE,NaN,NaN
147289,2021-10-22,GGZ000EC,Ubuntu Fuel Station,67.53,1167.70,FORD,FORD RANGER 2.2 TDCI XL PLUS 4,EDUCATION,MANAGED MAINTENANCE,NaN,NaN
35498,2021-05-19,GGX451EC,Straunway Service Station,45.00,703.35,VOLKSWAGEN,VW JETTA VI 1.6 TDI COMFORT,TRANSPORT,MANAGED MAINTENANCE,NaN,NaN
57845,2021-06-06,GGY290EC,Total Oxford East London,55.36,915.65,Nissan,NISSAN HARDBODY 2.5 TDI HIRIDE,EDUCATION,MANAGED MAINTENANCE,NaN,NaN
100732,2021-07-31,GGX436EC,Masepa Motors Cc,47.86,810.75,VOLKSWAGEN,VW POLO VIVO 1.6 SEDAN,TRANSPORT,MANAGED MAINTENANCE,NaN,NaN
70868,2021-07-16,GGF819EC,Total King,48.08,806.80,VOLKSWAGEN,POLO VIVO 1.6 HIGHLINE HATCH BACK,TRANSPORT,MANAGED MAINTENANCE,NaN,NaN
35583,2021-05-17,GGX501EC,Engen Lusikisiki Convenience,45.25,773.78,VOLKSWAGEN,VW POLO VIVO 1.6 SEDAN,TRANSPORT,MANAGED MAINTENANCE,NaN,NaN
178134,2021-12-01,GGF134EC,Ubuntu Fuel Station T/A Engen Bisho,67.06,1307.05,TOYOTA,HILUX 2.4GD-6SR 4X4 D/C,EDUCATION,MANAGED MAINTENANCE,NaN,NaN
197265,2022-01-16,GGF100EC,The Buffalo River One Stop,39.15,743.85,TOYOTA,TOYOTA COROLLA QUEST 1.6,EDUCATION,MANAGED MAINTENANCE,NaN,NaN


Most seem to be fine and operating, although the fleet register dataset is from a later date and the vehicles could have been placed in managed maintenance by that point.

## 3.7 District

In [122]:
# Display all unique District values
clean_data['District'].unique()

array(['OR Tambo', 'Sarah Baartman', 'Alfred Nzo', 'Joe Gqabi',
       'Amathole', 'Buffalo City', 'Chris Hani', nan,
       'Nelson Mandela Metro', 'Head Office', 'Nelson Mandela',
       'EMS College'], dtype=object)

In [123]:
# Check the distribution of the District variable
clean_data['District'].value_counts(dropna=False)

District
NaN                     73741
Sarah Baartman          17638
OR Tambo                16930
Chris Hani              13594
Amathole                12727
Joe Gqabi               12241
Alfred Nzo              11485
Buffalo City             8820
Nelson Mandela           4602
Nelson Mandela Metro     2505
Head Office              1291
EMS College               373
Name: count, dtype: int64

In [124]:
# Change the NA district to "UNKNOWN"
clean_data['District'] = clean_data['District'].fillna('UNKNOWN')

In [125]:
# Check the distribution of the District variable
clean_data['District'].value_counts(dropna=False)

District
UNKNOWN                 73741
Sarah Baartman          17638
OR Tambo                16930
Chris Hani              13594
Amathole                12727
Joe Gqabi               12241
Alfred Nzo              11485
Buffalo City             8820
Nelson Mandela           4602
Nelson Mandela Metro     2505
Head Office              1291
EMS College               373
Name: count, dtype: int64

## 3.8 Site

In [126]:
# Check the distribution of the Site variable
clean_data['Site'].value_counts(dropna=False)

Site
NaN                                    73617
Port Elizabeth                          3288
East London                             2897
EAST LONDON                             2794
Mthatha                                 2722
                                       ...  
Sundays Valley Hospital *P                12
PE Medical Depot *P                        9
Forensic S Port Alfred *P                  7
NQAMAKWE                                   6
Lilitha Nursing Col - All Saints *P        1
Name: count, Length: 320, dtype: int64

In [127]:
# Change the NA site values to "UNKNOWN"
clean_data['Site'] = clean_data['Site'].fillna('UNKNOWN')

In [128]:
# Convert all sites to upper case
clean_data['Site'] = clean_data['Site'].str.upper()

In [129]:
# Extract unique site names
unique_names = clean_data['Site'].unique()

# Remove null values
unique_names = unique_names[~pd.isnull(unique_names)]

# Dictionary to hold each name and its similar names
similar_names_dict = {}

# Iterate over each unique name
for name in unique_names:
    # Find similar names
    similar_names = find_similar_names(name, unique_names)
    # Store in the dictionary
    similar_names_dict[name] = similar_names

# Print the results, excluding exact matches
for name, similarities in similar_names_dict.items():
    if similarities:  # Only print if there are non-exact matches
        print(f"{name}: {similarities}")

ALIWAL NORTH: ['ALWAL NORTH']
NTABANKULU: ['TABANKULU']
GRAAFF REINET: ['GRAAF REINET', 'GRAAF-REINET']
BARKLYEAST: ['BARKLY EAST']
LADY GREY HOSPITAL *P: ['GREY HOSPITAL *P']
GRAAF REINET: ['GRAAFF REINET']
BARKLY EAST: ['BARKLYEAST']
GLEN GREY HOSPITAL *P: ['GREY HOSPITAL *P']
LILITHA NURSING COL - QUEENSTOWN (STERKSPRUIT) *P: ['LILITHA NURSING COL - QUEENSTOWN *P']
LILITHA NURSING COL - QUEENSTOWN *P: ['LILITHA NURSING COL - QUEENSTOWN (STERKSPRUIT) *P']
TABANKULU: ['NTABANKULU']
GREY HOSPITAL *P: ['LADY GREY HOSPITAL *P', 'GLEN GREY HOSPITAL *P']
FORENSIC BUTTERWORTH *P: ['FORENSIC S BUTTERWORTH *P']
ALWAL NORTH: ['ALIWAL NORTH']
FORENSIC S BUTTERWORTH *P: ['FORENSIC BUTTERWORTH *P']
GRAAF-REINET: ['GRAAFF REINET']


In [130]:
# Consolidate similar names into a single representative name for each group
names_map = consolidate_names(similar_names_dict)

# Replace names in the DataFrame
clean_data = replace_names(clean_data, names_map, var='Site')

# View the replaced DataFrame
clean_data.head()

,Transaction Date,REG_NUM,Merchant Name,No. of Litres,Transaction Amount,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,District
0,2021-04-01,GGA344EC,Ulundi Filling Station,43.06,695.45,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,LIBODE,OR Tambo
1,2021-04-01,GGA345EC,Sb Delta,31.02,478.17,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,JANSENVILLE,Sarah Baartman
2,2021-04-01,GGA345EC,Sb Delta,58.95,908.52,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,JANSENVILLE,Sarah Baartman
3,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,58.61,963.66,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,ALWAL NORTH,Alfred Nzo
4,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,60.02,986.76,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,ALWAL NORTH,Alfred Nzo


In [131]:
# Convert the names to camel case
clean_data['Site'] = clean_data['Site'].str.title()
clean_data.head()

,Transaction Date,REG_NUM,Merchant Name,No. of Litres,Transaction Amount,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,District
0,2021-04-01,GGA344EC,Ulundi Filling Station,43.06,695.45,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Libode,OR Tambo
1,2021-04-01,GGA345EC,Sb Delta,31.02,478.17,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Jansenville,Sarah Baartman
2,2021-04-01,GGA345EC,Sb Delta,58.95,908.52,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Jansenville,Sarah Baartman
3,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,58.61,963.66,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Alwal North,Alfred Nzo
4,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,60.02,986.76,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Alwal North,Alfred Nzo


In [132]:
def replace_site_names(df, site_replacements):
    """
    Replace site names in the dataframe based on certain patterns.

    :param df: DataFrame containing the 'Site' column.
    :return: DataFrame with updated 'Site' column.
    """

    # Temporarily fill NaN values with an empty string for comparison
    df['Site'] = df['Site'].fillna('')

    for old_site, new_site in site_replacements.items():
        # Check that the values are not null
        if not pd.isnull(old_site) and not pd.isnull(new_site):
            # Using str.startswith() to match the beginning of the string
            df.loc[df['Site'].str.startswith(old_site), 'Site'] = new_site

    # Revert the empty strings back to NaN
    df['Site'].replace('', pd.NA, inplace=True)

    return df

In [133]:
site_replacements = {
    '0': 'UNKNOWN',
    '43501': 'UNKNOWN'
}

# Apply the function to replace model names
clean_data = replace_site_names(clean_data, site_replacements)

# Example of the replaced DataFrame
clean_data.head()

,Transaction Date,REG_NUM,Merchant Name,No. of Litres,Transaction Amount,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,District
0,2021-04-01,GGA344EC,Ulundi Filling Station,43.06,695.45,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Libode,OR Tambo
1,2021-04-01,GGA345EC,Sb Delta,31.02,478.17,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Jansenville,Sarah Baartman
2,2021-04-01,GGA345EC,Sb Delta,58.95,908.52,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Jansenville,Sarah Baartman
3,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,58.61,963.66,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Alwal North,Alfred Nzo
4,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,60.02,986.76,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Alwal North,Alfred Nzo


In [134]:
# Check the final dimensions of the dataset
clean_data.shape

(175947, 11)

In [135]:
# Save the cleaned dataset to a new CSV file
clean_data.to_csv(os.path.join("..", "data", "cleaned_data.csv"), index=False)

# 4. Add the coordinates of the transactions based on the merchant name

In [68]:
# Read in the clean dataset 
clean_data = pd.read_csv(os.path.join("..", "data", "cleaned_data.csv"))

In [136]:
from geopy.geocoders import GoogleV3
import configparser
import pandas as pd

# Read API key from config file
config = configparser.ConfigParser()
config.read('../config.ini')
api_key = config['DEFAULT']['GOOGLE_API_KEY']

# Create a geocoder object
geolocator = GoogleV3(api_key=api_key)

# Function to get coordinates
def get_coordinates(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error occurred: {e}")
    return None, None

# Caching the geocoded results
cached_site_coordinates = {}

# Iterate over unique sites
for site in clean_data['Site'].unique():
    address = f"{site}, Eastern Cape, South Africa"
    if address not in cached_site_coordinates:
        cached_site_coordinates[address] = get_coordinates(address)

# Apply cached coordinates to the DataFrame
clean_data['Site Lat'] = clean_data['Site'].map(lambda x: cached_site_coordinates[f"{x}, Eastern Cape, South Africa"][0])
clean_data['Site Long'] = clean_data['Site'].map(lambda x: cached_site_coordinates[f"{x}, Eastern Cape, South Africa"][1])

In [137]:
clean_data.head()

,Transaction Date,REG_NUM,Merchant Name,No. of Litres,Transaction Amount,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,District,Site Lat,Site Long
0,2021-04-01,GGA344EC,Ulundi Filling Station,43.06,695.45,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Libode,OR Tambo,-31.538746,29.028448
1,2021-04-01,GGA345EC,Sb Delta,31.02,478.17,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Jansenville,Sarah Baartman,-32.943528,24.654919
2,2021-04-01,GGA345EC,Sb Delta,58.95,908.52,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Jansenville,Sarah Baartman,-32.943528,24.654919
3,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,58.61,963.66,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Alwal North,Alfred Nzo,-30.695841,26.708941
4,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,60.02,986.76,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Alwal North,Alfred Nzo,-30.695841,26.708941


In [138]:
# Caching the geocoded results
cached_coordinates = {}

# Iterate over unique sites
for merchant in clean_data['Merchant Name'].unique():
    address = f"{merchant}, Eastern Cape, South Africa"
    if address not in cached_coordinates:
        cached_coordinates[address] = get_coordinates(address)

# Apply cached coordinates to the DataFrame
clean_data['Merchant Lat'] = clean_data['Merchant Name'].map(lambda x: cached_coordinates[f"{x}, Eastern Cape, South Africa"][0])
clean_data['Merchant Long'] = clean_data['Merchant Name'].map(lambda x: cached_coordinates[f"{x}, Eastern Cape, South Africa"][1])

In [148]:
# Save the transformed data to a csv file in the data folder
clean_data.to_csv(os.path.join("..", "data", "cleaned_data_with_coords.csv"), index=False)

# 5. Add the true fuel prices to the transactions

## 5.1 Load the fuel price dataset

In [149]:
# Load the cleaned data with coordinates
clean_data = pd.read_csv(os.path.join("..", "data", "cleaned_data_with_coords.csv"))

## 5.2 Create a fuel type column to store each vehicles fuel type

In [140]:
# Check the unique vehicle model values
clean_data['MODEL DERIVATIVE'].unique()

array(['D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER',
       'D-MAX 250 HO 4X4 CREW CAB HI-RIDER',
       'VW CRAFTER 50 2.0 TDI 120 KW XLWB 23S',
       'HILUX DC 2.4GD6 4X4 SRX MT', 'HILUX DC 2.4GD6 RB SRX MT',
       'D-MAX 300 4X4 REGULAR CAB LX',
       'D-MAX 250C REGULAR CAB  FLEETSIDE  REGUL',
       'ISUZU D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER AMBULANCE',
       '250 D/C HI-RIDE GEN 6', 'TOYOTA QUANTUM 2.7 14 SEAT',
       'TOYOTA QUANTUM 2.5 D-4D SESFIK', 'VW GOLF & GTI 2.0 DSG 169 KW',
       'VW POLO VIVO 1.6 SEDAN', 'VW AMAROK 2.0 TRENDLINE - ACD7 D/C',
       'AMAROK 2.0 BI TDI 132 KW AUTO HIGHLINE 4 MOTION DC',
       'VW CRAFTER PANEL VAN AMBULANCE',
       'VW CRAFTER 50 2.0 TDI XLWB AMBULANCE',
       'VW CRAFTER 50 2.0 BITDI HR 120', 'NISSAN HARDBODY NP300 2.5 TDI',
       'NISSAN 2.5D SE+SC 4X4 ABS', 'NISSAN HARDBODY NP300 2.5TDI H',
       'NISSAN 2.4 DC 4X4 HR+ABS', 'NISSAN HARDBODY NP300 2.4I HI-',
       'NISSAN K36 NP300 2.4 HI RIDER D/C (RESPONSE)',
       'NISSAN N

In [150]:
# Get the unique Model Derivatives
unique_models = clean_data['MODEL DERIVATIVE'].unique()

# Save them to a CSV file
unique_models_df = pd.DataFrame(unique_models, columns=['Model Derivative'])

# Create a new column for fuel type and assign all values to "Diesel"
unique_models_df['Fuel Type'] = 'Diesel'

# Sort the DataFrame by the model derivative
unique_models_df = unique_models_df.sort_values(by='Model Derivative')

unique_models_df.to_csv(os.path.join("..", "data", "unique_models.csv"), index=False)

In [151]:
unique_models_df.shape

(229, 2)

In [154]:
petrol = ['CHEVROLET AVEO 1.6 L', 'FIAT UNO 1.2 5DR', 'FORD BANTAM 1.3I', 'FORD FIESTA 1.4I 5DR', 'FORD FIGO 1.4 AMBIENTE', 'FORD FOCUS 2.0 GDI TREND', 'FORD LASER 1.3 TRACER TONIC', 'FORD RANGER 2.5I P/U S/C', 'FORD RANGER 2.5I XL P/U D/C', 'GOLF 1.4 TSI COMFORTLINE DSG (92 KW) BQ13HZ', 'HYUNDAI ACCENT 1.6 FLUID 5DR', 'HYUNDAI ACCENT 1.6 FLUID AT 5D', 'HYUNDAI ACCENT 1.6 GL', 'HYUNDAI ACCENT 1.6 GLIDE', 'HYUNDAI ACCENT 1.6 GLS', 'HYUNDAI ACCENT 1.6 GLS A/T', 'HYUNDAI ACCENT 1.6 MANUAL', 'IKON 1.6 AMBIENTE',
          'ISUZU KB 160 FLEETSIDE', 'JEEP GRAND CHEROKEE 5.7 V8 O/L', 'M-BENZ GLE 500 4MATIC', 'MAHINDRA XYLO 2.5 CRDE E2 8 SE', 'NISSAN 1400 STD 5 SPEED P/U', 'NISSAN ALMERA 1.6 COMFORT', 'NISSAN HARDBODY 2.0I LWB (K08)', 'NISSAN HARDBODY 2.4I HIRIDER D', 'NISSAN HARDBODY 2.4I HR DC 4X4', 'NISSAN HARDBODY 2.4I LWB 4X4 P', 'NISSAN HARDBODY 2.4I LWB H/R K', 'NISSAN HARDBODY 2000 I LWB', 'NISSAN HARDBODY 2000I SWB', 'NISSAN HARDBODY 2400I 4X4', 'NISSAN HARDBODY 2400I LWB', 'NISSAN HARDBODY 2400I SE HIRID', 'NISSAN HARDBODY NP300 2.0I LWB', 'NISSAN HARDBODY NP300 2.4I 4X4', 'NISSAN HARDBODY NP300 2.4I HI-', 'NISSAN HARDBODY NP300 2.4I LWB', 'NISSAN NV350 2.5 PETROL P/VAN PVD3 (AMBULANCE)', 'NISSAN NV350 2.5I NARROW F/C P', 'NISSAN NV350 2.5I WIDE F/C P/V', 'NISSAN NV350 2.5PETROL', 'OPEL ASTRA 1.6 ESSENTIA', 'OPEL CORSA 1.4 COLOUR 3DR', 'OPEL CORSA CLASSIC 1.6 COMFORT', 'OPEL CORSA LITE PLUS A/C', 'OPEL CORSA UTILITY 1.8I', 'POLO VIVO 1.6 HIGHLINE HATCH BACK',
          'TATA INDICA 1.4 DLS', 'TATA INDICA 1.4 LSI', 'TOYOTA AVANZA 1.5 SX', 'TOYOTA COROLLA QUEST 1.6', 'TOYOTA COROLLA QUEST 1.6 A/T', 'TOYOTA ETIOS SEDAN 1.5 XI SD', 'TOYOTA PRADO VX 4.0 V6 A/T', 'TOYOTA QUANTUM 2.7 14 SEAT', 'TOYOTA QUANTUM 2.7 LWB P/V', 'TOYOTA QUANTUM 2.7 SESFIKILE 16S', 'TOYOTA YARIS T3 A/C H/B', 'VW CADDY KOMBI 1.6I', 'VW CITI CHICO 1400', 'VW CITI SPORT 1.4I', 'VW GOLF 2.0 TSI GTI DSG', 'VW GOLF VII 1.4 TSI COMFORTLIN', 'VW POLO 1.6 SEDAN', 'VW POLO 1.6 SEDAN AUTOMATIC', 'VW POLO VIVO 1.4', 'VW POLO VIVO 1.6 SEDAN', 'VW TENACITI 1.4I']
diesel = ['2009 TRANSPORTER', '250 D/C HI-RIDE GEN 6', '250 S/C SAFETY GEN 6', '250C S/CAB FLEETSIDE', '300 D/CAB LX 4X4', 'AMAROK 2.0 BI TDI 132 KW AUTO HIGHLINE 4 MOTION DC', 'AUDI 716-Q7 3.0D-183-QA8-7S', 'AUDI Q5 4.0L TDI QUATTRO', 'AUDI Q5 40TDI QUATTRO', 'AUDI Q7 3.0 TDI V6 QUATTRO TIP', 'B-VJ12 G02 X4 XDRIV 5 DOOR', 'BMW MOTORBIKE', 'BMW X4', 'BMW X4 XDRIVE 20D', 'BMW X4 XDRIVE 20D AT SAC', 'BMW X5 XDRIVE 30D SAV', 'CRAFTER 35 2.0 TDI 103 KW MAN MWB LCV CODE 8 (AMBULANCE)', 'CRAFTER 35 2.0 TDI 103KW MAN MWB', 'D-MAX 250 C/CAB 4X4', 'D-MAX 250 HO 4X4 CREW CAB HI-RIDER', 'D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER', 'D-MAX 250 HO CREW CAB HI-RIDER', 'D-MAX 250 HO CREW CAB HI-RIDER 4X2 LCV D/CAB M180TX1', 'D-MAX 250C FLEETSIDE REGULAR CAB', 'D-MAX 250C FLEETSIDE REGULAR CAB (M180TW1)', 'D-MAX 250C REGULAR CAB FLEETSIDE REGUL', 'D-MAX 300 4X4 REGULAR CAB LX', 'DISCOVERY SPORT 132 KW 7S AUTO D', 'FORD RANGER', 'FORD RANGER 2.2 TDCI D/CAB 5MT 4X2', 'FORD RANGER 2.2 TDCI D/CAB XL 6MT 4X4 (WITH CANOPY)', 'FORD RANGER 2.2 TDCI S/CAB LR 5MT 4X2', 'FORD RANGER 2.2 TDCI S/CAB XL 6MT 4X4 (AMBULANCE)', 'FORD RANGER 2.2 TDCI XL PLUS 4', 'FORD RANGER 2.2 XL D/CAB', 'FORD RANGER 2.2TDCI 4X2 S/CAB', 'FORD RANGER 2.2TDCI L/R P/U C/', 'FORD RANGER 2.2TDCI L/R P/U D/C', 'FORD RANGER 2.2TDCI L/R P/U S/', 'FORD RANGER 2.2TDCI P/U D/C', 'FORD RANGER 2.2TDCI XL 4X4 P/U', 'FORD RANGER 2.2TDCI XL P/U D/C', 'FORD RANGER 2.2TDCI XL P/U S/C', 'FORD RANGER 2.2TDCI XL P/U SUP', 'FORD RANGER 2.2TDCI XLS P/U S/', 'FORD RANGER 2200 LWB XL', 'FORD RANGER 3.2TDCI WILDTRACK', 'FORD RANGER 3.2TDCI XLS P/U S/', 'FORD RANGER D/C', 'FORD RANGER D/CAB', 'FORD RANGER D/CAB 4X4', 'FORD RANGER D/CAB XL 4X4HR', 'HARDBODY 2.7 LWB S/CAB 4X2', 'HILUX 2.4 GD 5MT A/C S/C', 'HILUX 2.4 GD-6 4X4 SR MT A12 S/C', 'HILUX 2.4 GD-6 RB SRX MT S/C', 'HILUX 2.4 GD-6 RB SRX MT S/C', 'HILUX 2.4 GD-6 RB SRX MT S/C', 'HILUX 2.4GD-6SR 4X4 D/C', 'HILUX DC 2.4GD6 4X4 SR MT', 'HILUX DC 2.4GD6 4X4 SRX MT', 'HILUX DC 2.4GD6 RB SRX MT', 'HILUX SC 2.0 VVTI S', 'HILUX SC 2.4 GD 5MT A/C', 'HILUX SC 2.4GD6 4X4 SR MT', 'HILUX SC 2.4GD6 RB SRX MT', 'HILUXDC 2.4GD6 RB SR MT', 'HILUXSC 2.4GD S A/C 5MT', 'INTERSTER 2.5 MWB PANEL VAN',
          'ISUZU D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER AMBULANCE', 'ISUZU D-MAX 250C REGULAR CAB FLEETSIDE', 'ISUZU KB 300 TDI FLEETSIDE', 'ISUZU KB 300 TDI LX', 'K40 NISSAN NP300 2.5 TDI LWB S/CAB', 'KB 250 D-TEQ HO LE PU D/', 'KB 250 HO 4X4 CREW CAB HI-RIDER', 'KB 250 HO 4X4 REGULAR CAB HI-RIDER', 'KB 250 HO CREW CAB HI-RIDER', 'KB 250 HO HI-RIDER CREW CAB', 'KB 250C BASE REGULAR CAB', 'KB 250C REGULAR CAB FLEETSIDE', 'LAND CRUSER PRADO', 'LAND ROVER DISCOVERY SPORT 132 KW 7S AUTO D', 'LANDCRUISER 79 PICK UP 4.2 DIESEL S/C', 'M-BENZ 309D/36CDI P/VAN SR 4X2', 'M-BENZ 416 CDI PV HR', 'M-BENZ GLE 350D 4MATIC', 'M-BENZ SPRINTER 515 CDI PV', 'M-BENZ VITO 113 CDI F/C P/V', 'NISSAN 2.4 DC 4X4 HR+ABS', 'NISSAN 2.5 4X4 S/C (MOBILE CLINIC)', 'NISSAN 2.5D SE+SC 4X4 ABS', 'NISSAN HARDBODY', 'NISSAN HARDBODY 2.5 TDI HIRIDE', 'NISSAN HARDBODY 2.5 TDI LWB S', 'NISSAN HARDBODY 2.5 TDI LWB SE', 'NISSAN HARDBODY 2.5TDI LWB K03', 'NISSAN HARDBODY 2700 D 4X2 D/C', 'NISSAN HARDBODY 2700D LWB', 'NISSAN HARDBODY NP300 2.5 TDI', 'NISSAN HARDBODY NP300 2.5TDI H', 'NISSAN INTERSTAR 2.5 DCI SR', 'NISSAN K36 NP300 2.4 HI RIDER D/C (RESPONSE)', 'NISSAN NP200 1.6 A/C SAFETY P', 'NISSAN NV350 2.5 16 SEAT IMPEN', 'NISSAN PATROL 3.0 TDI 4X4 P/U', 'NISSAN PATROL 3.0DI GL', 'NP300 2.0 SC 4X2', 'QUANTUM 2.7 10S COMMUTER',
          'RANGER 2.2 D/C XL 6MT 4X2', 'RANGER 2.2 TDC I DOUBLE CAB XL 6MT', 'RANGER 2.2 TDCI XL 6MT 4X2 S/C', 'RANGER 2.2 XL D/C 4X4', 'RANGER 2.2D 118KW 6MT 4X4 HR D/CAB', 'RANGER 2.2D XL 6MT 4X2 S/C', 'RANGER 2019 5MY D/C XL 2.2D 118KW 6AT 4X2HR', 'RANGER 2019 5MY D/C XL 2.2D 118KW 6AT 4X4HR', 'RANGER 2019 5MY REGULAR CAB XL 2.2D 118 KW 6MT 4X4HR', 'RANGER 2019 MY DBL CAB XL 2.2D118KW 6MT 4X4HR - RESPONSE', 'RANGER D/C 4X4HR', 'RANGER DC 2.2D 4X2 5MT 88KW', 'RANGER REGULAR CAB 4X2 HR', 'REG CAB XL 2.2D 118 KW 6MT 4X2 HR', 'TOYOTA FORTUNER 2.4 GD-6 RB AT', 'TOYOTA HILUX 2.0 VVTI P/U S/C', 'TOYOTA HILUX 2.4 GD 5MT A/C S/C', 'TOYOTA HILUX 2.4 GD-6 RB SRX', 'TOYOTA HILUX 2.4 GD-6 SR 4X4', 'TOYOTA HILUX 2.4 GD-6 SRX 4X4', 'TOYOTA HILUX 2.4GD-6SR 4X4 DC', 'TOYOTA HILUX 2.5 D-4D P/U S/C', 'TOYOTA HILUX 2.5 D-4D SRX 4X4', 'TOYOTA HILUX 2.5 D-4D SRX R/B', 'TOYOTA HILUX 2200 LWB P/U (P)', 'TOYOTA HILUX 4X4 D/CAB', 'TOYOTA HILUX S/C 2.4 GD BMT A/C', 'TOYOTA LAND CRUISER 79 PICK UP 4.2 DIESEL S/C', 'TOYOTA QUANTUM 2.5 D-4D LWB PV', 'TOYOTA QUANTUM 2.5 D-4D SESFIK', 'VN751-PANEL VAN 1.9D-077-FM5-LN30', 'VW AMAROK 2.0 TRENDLINE - ACD7 D/C', 'VW AMAROK 2.0TDI TREND', 'VW CADDY 2.0TDI (81KW) F/C C/C', 'VW CADDY MAXI 2.0TDI (81KW)', 'VW CRAFTER', 'VW CRAFTER 35 2.0 TDI 80KW F/C', 'VW CRAFTER 35 2.0 TDI MWB 16 SEATER', 'VW CRAFTER 35 2.0 TDI MWB AMBULANCE', 'VW CRAFTER 35. 20 TDI MWB AMBULANCE', 'VW CRAFTER 50 2.0 BITDI HR 120', 'VW CRAFTER 50 2.0 TDI 120 KW XLWB 23S', 'VW CRAFTER 50 2.0 TDI 120 KW XLWB PANEL VAN', 'VW CRAFTER 50 2.0 TDI HR 80KW', 'VW CRAFTER 50 2.0 TDI XLWB 23 SEATER', 'VW CRAFTER 50 2.0 TDI XLWB AMBULANCE', 'VW CRAFTER PANEL VAN', 'VW CRAFTER PANEL VAN AMBULANCE', 'VW GOLF & GTI 2.0 DSG 169 KW', 'VW JETTA VI 1.6 TDI COMFORT', 'VW VN-AMAROK 2.0D-103-HM6-282']

# Check the length of the petrol and diesel lists
len(petrol) + len(diesel)

229

In [155]:
# Create a new fuel type column in clean_data
clean_data['Fuel Type'] = 'Diesel'

# Replace the fuel type for the petrol vehicles
clean_data.loc[clean_data['MODEL DERIVATIVE'].isin(petrol), 'Fuel Type'] = 'Petrol'

In [156]:
# Save the transformed data to a csv file in the data folder
clean_data.to_csv(os.path.join("..", "data", "cleaned_data_with_coords_and_fuel_type.csv"), index=False)

## Add the actual fuel prices

In [157]:
# Read in the dataset
clean_data = pd.read_csv(os.path.join("..", "data", "cleaned_data_with_coords_and_fuel_type.csv"))

# Convert the transaction date to datetime
clean_data['Transaction Date'] = pd.to_datetime(clean_data['Transaction Date'])

In [158]:
file_path_fuel = os.path.join("..", "data", "FuelPricesWithDates.csv")

# Load the dataset
fuel_data = pd.read_csv(file_path_fuel, delimiter=';').transpose()


# Resetting the index to make the first row as header
fuel_data.reset_index(inplace=True)
new_header = fuel_data.iloc[0] # grab the first row for the header
fuel_data = fuel_data[1:] # take the data less the header row
fuel_data.columns = new_header # set the header row as the dataframe header


# Rename the columns
fuel_data.rename(columns={'95 LRP (c/l)': 'Petrol',
                          'Diesel 0.05% (c/l) ': 'Diesel',
                          '95 ULP (c/l) *': 'Petrol Inland',
                          'Diesel 0.05% (c/l) **': 'Diesel Inland'}, inplace=True)

# Change the Date column to datetime
fuel_data['Date'] = pd.to_datetime(fuel_data['Date'])

# Displaying the first few rows of the transposed dataset
fuel_data.head()

,Date,Petrol,Diesel,Petrol Inland,Diesel Inland
1,2020-12-02,1376,"1185,12",1446,"1245,42"
2,2021-01-06,1416,"1240,12",1486,"1300,42"
3,2021-02-03,1497,"1298,12",1567,"1358,42"
4,2021-03-03,1562,"1352,12",1632,"1412,42"
5,2021-04-07,1660,"1416,12",1732,"1477,62"


In [159]:
# Swap out all ',' for '.' in the 'Petrol' and 'Diesel' column
fuel_data['Petrol'] = fuel_data['Petrol'].str.replace(',', '.').str.strip()
fuel_data['Diesel'] = fuel_data['Diesel'].str.replace(',', '.').str.strip()
fuel_data['Petrol Inland'] = fuel_data['Petrol Inland'].str.replace(',', '.').str.strip()
fuel_data['Diesel Inland'] = fuel_data['Diesel Inland'].str.replace(',', '.').str.strip()

# Convert the columns to the correct format
fuel_data['Date'] = pd.to_datetime(fuel_data['Date'])
fuel_data['Petrol'] = pd.to_numeric(fuel_data['Petrol']) / 100
fuel_data['Diesel'] = pd.to_numeric(fuel_data['Diesel']) / 100
fuel_data['Petrol Inland'] = pd.to_numeric(fuel_data['Petrol Inland']) / 100
fuel_data['Diesel Inland'] = pd.to_numeric(fuel_data['Diesel Inland']) / 100

In [160]:
fuel_data

,Date,Petrol,Diesel,Petrol Inland,Diesel Inland
1,2020-12-02,13.76,11.8512,14.46,12.4542
2,2021-01-06,14.16,12.4012,14.86,13.0042
3,2021-02-03,14.97,12.9812,15.67,13.5842
4,2021-03-03,15.62,13.5212,16.32,14.1242
5,2021-04-07,16.60,14.1612,17.32,14.7762
6,2021-05-05,16.51,13.8512,17.23,14.4662
7,2021-06-02,16.41,14.0512,17.13,14.6662
8,2021-07-07,16.67,14.4712,17.39,15.0862
9,2021-08-04,17.58,15.0270,18.30,15.6420
10,2021-09-01,17.62,14.8748,18.34,15.4898


In [166]:
def get_fuel_price(transaction_date, fuel_type, fuel_prices_df, inland=False):
    # Filter the prices up to the transaction date
    relevant_prices = fuel_prices_df[fuel_prices_df['Date'] <= transaction_date].iloc[-1]
    
    if inland:
        if fuel_type == 'Petrol':
            return relevant_prices['Petrol Inland']
        else:
            return relevant_prices['Diesel Inland']

    # Select the appropriate fuel price based on vehicle category
    if fuel_type == 'Petrol':
        return relevant_prices['Petrol']
    else:
        return relevant_prices['Diesel']

In [167]:
clean_data['Coastal Petrol'] = clean_data.apply(
    lambda row: get_fuel_price(row['Transaction Date'], 'Petrol', fuel_data), 
    axis=1
)

clean_data['Inland Petrol'] = clean_data.apply(
    lambda row: get_fuel_price(row['Transaction Date'], 'Petrol', fuel_data, inland=True), 
    axis=1
)

clean_data['Coastal Diesel'] = clean_data.apply(
    lambda row: get_fuel_price(row['Transaction Date'], 'Diesel', fuel_data), 
    axis=1
)

clean_data['Inland Diesel'] = clean_data.apply(
    lambda row: get_fuel_price(row['Transaction Date'], 'Diesel', fuel_data, inland=True), 
    axis=1
)

clean_data.head()

,Transaction Date,REG_NUM,Merchant Name,No. of Litres,Transaction Amount,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,RATE CARD CATEGORY,Site,...,Merchant Lat,Merchant Long,Fuel Type,Actual Fuel Price,Actual Fuel Price Inland,Estimated Price Per Litre,Coastal Petrol,Inland Petrol,Coastal Diesel,Inland Diesel
0,2021-04-01,GGA344EC,Ulundi Filling Station,43.06,695.45,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Libode,...,-30.627737,28.226245,Diesel,13.5212,14.1242,16.150720,15.62,16.32,13.5212,14.1242
1,2021-04-01,GGA345EC,Sb Delta,31.02,478.17,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Jansenville,...,-32.296840,26.419389,Diesel,13.5212,14.1242,15.414894,15.62,16.32,13.5212,14.1242
2,2021-04-01,GGA345EC,Sb Delta,58.95,908.52,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Jansenville,...,-32.296840,26.419389,Diesel,13.5212,14.1242,15.411705,15.62,16.32,13.5212,14.1242
3,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,58.61,963.66,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Alwal North,...,-30.691447,28.505138,Diesel,13.5212,14.1242,16.441904,15.62,16.32,13.5212,14.1242
4,2021-04-01,GGA346EC,Mount Fletcher Motors Eastern Cape,60.02,986.76,ISUZU,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,EMERGENCY MEDICAL SERVICES,CATEGORY 8: LDV Ambulance,Alwal North,...,-30.691447,28.505138,Diesel,13.5212,14.1242,16.440520,15.62,16.32,13.5212,14.1242


In [168]:
# Create the estimated price per litre variable
clean_data['Estimated Price Per Litre'] = clean_data['Transaction Amount'] / clean_data['No. of Litres']

In [173]:
# Display the Estimated ppl and the Actual ppl and inland ppl
clean_data[['Transaction Date', 'Fuel Type', 'MODEL DERIVATIVE',
            'Estimated Price Per Litre', 'Coastal Petrol', 'Inland Petrol', 'Coastal Diesel', 'Inland Diesel']].sample(15)

,Transaction Date,Fuel Type,MODEL DERIVATIVE,Estimated Price Per Litre,Coastal Petrol,Inland Petrol,Coastal Diesel,Inland Diesel
43529,2021-06-24,Diesel,NISSAN K36 NP300 2.4 HI RIDER D/C (RESPONSE),16.580359,16.41,17.13,14.0512,14.6662
144850,2021-12-07,Diesel,D-MAX 250 HO 4X4 REGULAR CAB HI-RIDER,19.323496,19.57,20.29,17.3098,17.9248
42219,2021-06-16,Petrol,NISSAN NV350 2.5PETROL,16.673247,16.41,17.13,14.0512,14.6662
148693,2021-12-10,Petrol,TOYOTA COROLLA QUEST 1.6,19.842970,19.57,20.29,17.3098,17.9248
51852,2021-07-02,Diesel,VW CRAFTER 50 2.0 TDI XLWB AMBULANCE,16.031431,16.41,17.13,14.0512,14.6662
37201,2021-06-17,Petrol,VW POLO VIVO 1.6 SEDAN,16.682294,16.41,17.13,14.0512,14.6662
116234,2021-10-27,Petrol,VW POLO VIVO 1.6 SEDAN,18.147409,17.61,18.33,15.1028,15.7178
108200,2021-10-22,Petrol,HYUNDAI ACCENT 1.6 GL,18.380408,17.61,18.33,15.1028,15.7178
57108,2021-07-05,Petrol,VW POLO VIVO 1.6 SEDAN,16.580344,16.41,17.13,14.0512,14.6662
168549,2022-01-19,Diesel,D-MAX 250C REGULAR CAB FLEETSIDE REGUL,19.081509,18.89,19.61,16.6318,17.2468


In [172]:
# Save the data frame
clean_data.to_csv(os.path.join("..", "data", "Final with Coords, Fuel Type and Prices.csv"), index=False)

## Create a fuel efficiency dataset based on tracker data (too many issues)

In [1]:
import pandas as pd
import os

# Read in the dataset
df = pd.read_csv(os.path.join("..", "data", "Final with Coords, Fuel Type and Prices.csv"))

In [2]:
# Read in the tracker data
file_path = os.path.join("..", "data", "OdoValuesTracker.csv")

# Load the dataset
tracker_data = pd.read_csv(file_path)

In [3]:
tracker_data.head()

,Reg,Date,Odo,CastedOdo
0,GGF356EC,2021-04-01,122 148,122148.0
1,GGF360EC,2021-04-01,25 600,25600.0
2,GGF364EC,2021-04-01,14 135,14135.0
3,GGF367EC,2021-04-01,64 839,64839.0
4,GGF369EC,2021-04-01,86 982,86982.0


In [19]:
# Check the number of unique registration numbers
tracker_data['Reg'].nunique()

1182

In [4]:
# Drop the Odo column and rename the CastedOdo to be Odo
tracker_data = tracker_data.drop(columns=['Odo']).rename(columns={'CastedOdo': 'Odo'})

In [5]:
df.columns

Index(['Transaction Date', 'REG_NUM', 'Merchant Name', 'No. of Litres',
       'Transaction Amount', 'VEHICLE MAKE', 'MODEL DERIVATIVE', 'DEPARTMENT',
       'RATE CARD CATEGORY', 'Site', 'District', 'Site Lat', 'Site Long',
       'Merchant Lat', 'Merchant Long', 'Fuel Type',
       'Estimated Price Per Litre', 'Coastal Petrol', 'Inland Petrol',
       'Coastal Diesel', 'Inland Diesel', 'Month Name', 'Weekday Name'],
      dtype='object')

In [8]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

# Group by 'REG_NUM' to get sum of 'No. of Litres' and date range
litres_data = df.groupby('REG_NUM').agg({'Transaction Date': ['min', 'max'],
                                        'VEHICLE MAKE': 'first',
                                        'MODEL DERIVATIVE': 'first',
                                        'DEPARTMENT': 'first',
                                        'District': 'first',
                                        'Site': 'first',
                                        'Site Lat': 'first',
                                        'Site Long': 'first',
                                        'Fuel Type': 'first',
                                        'RATE CARD CATEGORY': 'first',
                                        'Transaction Amount': ['sum', 'mean'],
                                        'No. of Litres': ['sum', 'mean']}).reset_index()

# Flatten the MultiIndex columns
litres_data.columns = ['REG_NUM', 'Min Date', 'Max Date', 'VEHICLE MAKE', 'MODEL DERIVATIVE', 'DEPARTMENT', 'District', 
                       'Site', 'Site Lat', 'Site Long', 'Fuel Type', 'RATE CARD CATEGORY',
                       'Total Transaction Amount', 'Mean Transaction Amount', 'Total No. of Litres', 'Mean No. of Litres']

In [9]:
litres_data.head()

,REG_NUM,Min Date,Max Date,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,District,Site,Site Lat,Site Long,Fuel Type,RATE CARD CATEGORY,Total Transaction Amount,Mean Transaction Amount,Total No. of Litres,Mean No. of Litres
0,GGA001EC,2021-04-04,2021-10-14,FORD,FORD RANGER 2.2TDCI L/R P/U C/,EMERGENCY MEDICAL SERVICES,OR Tambo,Umtata,-31.594050,28.756873,Diesel,8: LDV Ambulance,113281.58,803.415461,6770.20,48.015603
1,GGA005EC,2021-04-01,2021-11-08,FORD,FORD RANGER 2.2TDCI L/R P/U C/,EMERGENCY MEDICAL SERVICES,OR Tambo,Tsolo,-31.319447,28.754782,Diesel,8: LDV Ambulance,89670.40,896.704000,5167.84,51.678400
2,GGA007EC,2021-09-20,2021-11-25,FORD,FORD RANGER 2.2TDCI L/R P/U C/,EMERGENCY MEDICAL SERVICES,Unknown,Unknown,-32.296840,26.419389,Diesel,8: LDV Ambulance,2400.00,1200.000000,135.84,67.920000
3,GGA008EC,2021-04-01,2022-01-27,FORD,FORD RANGER 2.2TDCI L/R P/U C/,EMERGENCY MEDICAL SERVICES,Amathole,Butterworth,-32.325796,28.169376,Diesel,8: LDV Ambulance,65009.68,792.800976,3833.79,46.753537
4,GGA011EC,2021-04-05,2022-01-21,FORD,FORD RANGER 2.2TDCI L/R P/U C/,EMERGENCY MEDICAL SERVICES,Nelson Mandela,PE,-33.745241,25.568108,Diesel,8: LDV Ambulance,77063.50,975.487342,4580.87,57.985696


In [10]:
import pandas as pd

# Convert 'Date' to datetime in both datasets
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
tracker_data['Date'] = pd.to_datetime(tracker_data['Date'])

# Ensure consistent column names for vehicle registration
df.rename(columns={'REG_NUM': 'Reg'}, inplace=True)

# Calculate the min and max dates for each vehicle in both datasets
odo_range = tracker_data.groupby('Reg')['Date'].agg(['min', 'max']).reset_index()
litres_range = df.groupby('Reg')['Transaction Date'].agg(['min', 'max']).reset_index()

# Merge to find overlapping dates
overlap_data = pd.merge(odo_range, litres_range, on='Reg', how='inner', suffixes=('_odo', '_litres'))

# Determine the largest overlapping range
overlap_data['Overlap_Start'] = overlap_data[['min_odo', 'min_litres']].max(axis=1)
overlap_data['Overlap_End'] = overlap_data[['max_odo', 'max_litres']].min(axis=1)

# Filter out non-overlapping entries
overlap_data = overlap_data[overlap_data['Overlap_Start'] <= overlap_data['Overlap_End']]

# Initialize columns for total calculations
overlap_data['Total_Km'] = 0
overlap_data['Total_Litres'] = 0

# Calculate total Km and Litres for each vehicle
for index, row in overlap_data.iterrows():
    reg = row['Reg']
    start, end = row['Overlap_Start'], row['Overlap_End']

    # Filter for each vehicle within the overlapping dates
    odo_filtered = tracker_data[(tracker_data['Reg'] == reg) & (tracker_data['Date'] >= start) & (tracker_data['Date'] <= end)]
    litres_filtered = df[(df['Reg'] == reg) & (df['Transaction Date'] >= start) & (df['Transaction Date'] <= end)]

    # Calculate total kilometers and liters
    if not odo_filtered.empty and not litres_filtered.empty:
        total_km = odo_filtered['Odo'].iloc[-1] - odo_filtered['Odo'].iloc[0]
        total_litres = litres_filtered['No. of Litres'].sum()

        # Update the overlap_data DataFrame
        overlap_data.at[index, 'Total_Km'] = total_km
        overlap_data.at[index, 'Total_Litres'] = total_litres

# Calculate KMPL
overlap_data['KMPL'] = overlap_data['Total_Km'] / overlap_data['Total_Litres']


In [11]:

# Final dataset with overlapping date ranges and KMPL
final_data = overlap_data[['Reg', 'Overlap_Start', 'Overlap_End', 'Total_Km', 'Total_Litres', 'KMPL']].dropna()

final_data.head()


,Reg,Overlap_Start,Overlap_End,Total_Km,Total_Litres,KMPL
0,GGF355EC,2021-07-01,2021-12-03,8968,748.52,11.980976
1,GGF356EC,2021-04-01,2022-01-25,18420,1681.21,10.956395
2,GGF358EC,2021-07-01,2021-12-31,11320,1145.65,9.880854
3,GGF363EC,2021-07-01,2021-12-31,8571,822.54,10.420162
4,GGF364EC,2021-04-07,2021-06-30,8016,754.15,10.629185


In [12]:
final_data.shape

(902, 6)

In [13]:
litres_data.columns

Index(['REG_NUM', 'Min Date', 'Max Date', 'VEHICLE MAKE', 'MODEL DERIVATIVE',
       'DEPARTMENT', 'District', 'Site', 'Site Lat', 'Site Long', 'Fuel Type',
       'RATE CARD CATEGORY', 'Total Transaction Amount',
       'Mean Transaction Amount', 'Total No. of Litres', 'Mean No. of Litres'],
      dtype='object')

In [14]:
# Join the litres data to the final data
final_data = final_data.merge(litres_data[['REG_NUM', 'VEHICLE MAKE', 'MODEL DERIVATIVE',
       'DEPARTMENT', 'District', 'Site', 'Site Lat', 'Site Long', 'Fuel Type',
       'RATE CARD CATEGORY', 'Total Transaction Amount',
       'Mean Transaction Amount', 'Total No. of Litres', 'Mean No. of Litres']], 
                       left_on='Reg', right_on='REG_NUM', how='left')

In [15]:
final_data.head()

,Reg,Overlap_Start,Overlap_End,Total_Km,Total_Litres,KMPL,REG_NUM,VEHICLE MAKE,MODEL DERIVATIVE,DEPARTMENT,District,Site,Site Lat,Site Long,Fuel Type,RATE CARD CATEGORY,Total Transaction Amount,Mean Transaction Amount,Total No. of Litres,Mean No. of Litres
0,GGF355EC,2021-07-01,2021-12-03,8968,748.52,11.980976,GGF355EC,TOYOTA,TOYOTA COROLLA QUEST 1.6,SOCIAL DEVELOPMENT,Unknown,Unknown,-32.296840,26.419389,Petrol,1: Sedan 1400-1600cc,25911.94,647.798500,1506.14,37.653500
1,GGF356EC,2021-04-01,2022-01-25,18420,1681.21,10.956395,GGF356EC,TOYOTA,TOYOTA COROLLA QUEST 1.6,HEALTH,Alfred Nzo,Tb:Khotsong Santa Hospital *P,-32.777673,26.657569,Petrol,1: Sedan 1400-1600cc,30886.45,630.335714,1681.21,34.310408
2,GGF358EC,2021-07-01,2021-12-31,11320,1145.65,9.880854,GGF358EC,TOYOTA,HILUX SC 2.4GD6 4X4 SR MT,HEALTH,Head Office,Cd: Supply Chain Mangt *P,-32.296840,26.419389,Diesel,11: LDV 4x4 1 ton light,42156.61,936.813556,2537.83,56.396222
3,GGF363EC,2021-07-01,2021-12-31,8571,822.54,10.420162,GGF363EC,TOYOTA,TOYOTA COROLLA QUEST 1.6,SOCIAL DEVELOPMENT,Unknown,Unknown,-32.296840,26.419389,Petrol,1: Sedan 1400-1600cc,29794.70,692.900000,1695.23,39.423953
4,GGF364EC,2021-04-07,2021-06-30,8016,754.15,10.629185,GGF364EC,TOYOTA,TOYOTA COROLLA QUEST 1.6,SOCIAL DEVELOPMENT,Unknown,Unknown,-32.296840,26.419389,Petrol,1: Sedan 1400-1600cc,36452.06,639.509825,2007.25,35.214912


In [16]:
# Save the final data
final_data.to_csv(os.path.join("..", "data", "Final KMPL dataset.csv"), index=False)